In [5]:
%%capture --no-stderr
%pip install langchain langchain-community langchain-experimental langchain-text-splitters langchain-openai pydantic langchain-chroma langchain-tavily wikipedia gradio arxiv pymupdf pypdf

In [6]:
from getpass import getpass
import os

api_keys = ["OPENAI_API_KEY", "TAVILY_API_KEY"]
for key in api_keys:
    os.environ[key] = getpass(f"Enter your {key}:")

# Lab 2: Building a simple Research Agent

## Part A: Using the `PlanAndExecute` pre-built chain

In this first section of the lab, we will implement the `PlanAndExecute` chain provided by Langchain and run it in our Gradio chat UI. We'll identify the strengths and weaknesses of this version, then look at implementing our own after!

### Build the agent toolkit

In [7]:
from langchain_tavily import TavilySearch
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_core.tools import tool
from typing import Dict, Any, List
from langchain_openai import ChatOpenAI
import time

In [8]:
# Tavily Search client
tavily = TavilySearch(
    max_results = 5,
    topic = "general"
)

# Wikipedia client
wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())

In [9]:
@tool(description="A tool that performs general web search using Tavily search engine", parse_docstring=True)
def tavily_search(query: str) -> Dict[str, Any]:
  """This tool performs a search using the Tavily Search engine.
  It is useful for finding information on a given topic or query from
  a number of sources across the web.

  Args:
    query: The query to search for
  """
  return tavily.run(query)

@tool(description="A tool that performs a search on Wikipedia using the Wikipedia API", parse_docstring=True)
def wikipedia_search(query: str) -> Dict[str, Any]:
  """This tool performs a search on Wikipedia using the Wikipedia API. It is useful for finding information on a given topic or query from Wikipedia.

  Args:
    query: The query to search for
  """
  return wikipedia.run(query)

In [10]:
toolkit = [tavily_search, wikipedia_search]

### Set up the agent chains



In [11]:
from langchain_experimental.plan_and_execute.agent_executor import PlanAndExecute
from datetime import datetime

In [12]:
#
planner_llm = ChatOpenAI(model="gpt-4.1-nano")
executor_llm = ChatOpenAI(model="gpt-4.1-mini")

### (Optional) Add a system prompt

**NOTE:** For the Plan and Execute agent, our system prompt must help the Planner chain output a sequence of steps that can then be parsed by the Executor chain. In order to do this, we must explicitly write these instructions into the prompt:

In [13]:
planner_prompt = """
You are a planner who is an expert at coming up with a plan to
answer the following questions as best you understand. Think things
through step-by-step and output your step-by-step plan as a sequence of steps.
Use the tools that you have at your disposal to answer questions.
"""

In [14]:
from langchain_experimental.plan_and_execute.planners.chat_planner import load_chat_planner
from langchain_experimental.plan_and_execute.executors.agent_executor import load_agent_executor


planner = load_chat_planner(
    llm=planner_llm,
    system_prompt=planner_prompt,
)

executor = load_agent_executor(
    llm=executor_llm,
    tools=toolkit,
    verbose=True,
)

agent = PlanAndExecute(
    executor=executor,
    planner=planner,
    verbose=True,
    include_run_info=True
)

### Set up the UI

We'll use Gradio for our chat UI just like we did in our first lab:

In [15]:
def research_chat(question: str, history: List[Dict[str, Any]]):
  response = agent.invoke({"input": question, "history": history})
  return response.get("output", "No response from agent")

In [16]:
import gradio as gr

gr.ChatInterface(fn=research_chat, title="Research Agent Chat", type="messages").launch(debug=True)

c:\Users\Marcus\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


TypeError: ChatInterface.__init__() got an unexpected keyword argument 'type'

### Current Limitations

Out of the box, we get a ReAct style agent with rudimentary planning that can be used as a base for agentic applications. However, there are limitations to this implementation:

- Blackbox planning: The planning layer is mostly obfuscated from us
- Little control over agent behavior via prompts
- Little to no control over the logging
- Missing token usage or other useful metadata
- Documentation is sparse

While this might be ok for prototyping or very minimal applications, it is too brittle for anything substantial. The good thing is that we've learned enough to implement our OWN version of the Plan and Execute, ReAct style agent!

We are going to do exactly that in the next part of this lab.

## Part B: Implementing the Plan and Execute agent chain

**Instructions:**

We will build on what we've put together so far and, instead of using LangChain's implementations, we will build out a planner and executor, each as a modular chain. Then we will compose them together to create our ReAct style research agent with access to the Wikipedia and Tavily Search tools.

### The Planner Chain

In [ ]:
# === Planner Chain ===
# Converts a natural-language task into a structured JSON plan.

# %pip install -q langchain langchain-openai pydantic

import os
from typing import List, Optional, Literal
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.output_parsers import PydanticOutputParser

# ----- schema -----
class PlanStep(BaseModel):
    id: int = Field(..., description="1-based step index")
    action: Literal["search","read","call_api","run_code","query_db","summarize","analyze","visualize","other"]
    description: str
    tool: Optional[str] = Field(None, description="explicit tool name if any")
    inputs: dict = Field(default_factory=dict)
    success_criteria: str

class Plan(BaseModel):
    task: str
    assumptions: List[str] = []
    steps: List[PlanStep]
    final_artifact: str
    notes: Optional[str] = None

parser = PydanticOutputParser(pydantic_object=Plan)

MODEL_NAME = os.getenv("PLANNER_MODEL","gpt-4o-mini")
llm_plan = ChatOpenAI(model=MODEL_NAME, temperature=0)

system_txt = (
    "You are a precise AI Planner. Break the user's task into 3–7 reliable steps. "
    "Specify tools and inputs whenever external actions are needed. "
    "Do not invent credentials or endpoints; write assumptions instead. "
    "Return ONLY valid JSON that matches the schema."
)
prompt_plan = ChatPromptTemplate.from_messages([
    ("system", system_txt),
    ("human", "Task:\n{task}\n\nContext:\n{context}\n\n{format_instructions}")
])
fmt = parser.get_format_instructions()

planner_chain = prompt_plan | llm_plan | parser

def plan_task(task: str, context: str = "") -> Plan:
    return planner_chain.invoke({"task": task.strip(), "context": context.strip(), "format_instructions": fmt})


{
  "task": "Get hourly gym check-ins from Postgres, summarize peaks, and plot a bar chart.",
  "assumptions": [
    "Database credentials are stored in environment variables.",
    "The database contains a table with gym check-in records including timestamps."
  ],
  "steps": [
    {
      "id": 1,
      "action": "query_db",
      "description": "Connect to the Postgres database and retrieve hourly gym check-in data.",
      "tool": "Postgres",
      "inputs": {
        "query": "SELECT date_trunc('hour', check_in_time) AS hour, COUNT(*) AS check_in_count FROM gym_check_ins GROUP BY hour ORDER BY hour;"
      },
      "success_criteria": "Hourly check-in data is successfully retrieved."
    },
    {
      "id": 2,
      "action": "summarize",
      "description": "Analyze the retrieved data to identify peak check-in hours.",
      "tool": null,
      "inputs": {
        "data": "Hourly check-in data from step 1."
      },
      "success_criteria": "Peak hours are identified and summa

### The Executor Chain

In [18]:
# === Executor Chain ===
# Executes the steps from the planner with a small tool registry.

import os, time, json, traceback
from dataclasses import dataclass, field
from typing import Any, Dict, Optional, Callable, List

# optional deps
try:
    import requests
except Exception:
    requests = None

try:
    import psycopg  # psycopg3
except Exception:
    psycopg = None

from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

@dataclass
class StepResult:
    step_id: int
    action: str
    tool: Optional[str]
    ok: bool
    output: Any = None
    error: Optional[str] = None
    latency_s: float = 0.0

@dataclass
class ExecutionLog:
    task: str
    steps: List[StepResult] = field(default_factory=list)
    def add(self, s: StepResult): self.steps.append(s)
    @property
    def ok(self): return all(s.ok for s in self.steps)
    def to_json(self) -> str:
        return json.dumps({"task": self.task, "ok": self.ok, "steps":[s.__dict__ for s in self.steps]}, indent=2)

# ---- tools ----
def t_call_api(inp: Dict[str,Any]) -> Any:
    assert requests is not None, "requests not installed"
    method = (inp.get("method") or "GET").upper()
    r = requests.request(method, inp["url"], headers=inp.get("headers"), params=inp.get("params"),
                         json=inp.get("json"), data=inp.get("data"), timeout=float(inp.get("timeout",30)))
    return {"status": r.status_code, "headers": dict(r.headers), "text": r.text[:20000]}

def t_query_db(inp: Dict[str,Any]) -> Any:
    assert psycopg is not None, "psycopg not installed"
    dsn = dict(
        host=os.getenv("PGHOST","postgres"),
        port=int(os.getenv("PGPORT",5432)),
        dbname=os.getenv("PGDATABASE","n8n"),
        user=os.getenv("PGUSER","kura"),
        password=os.getenv("PGPASSWORD","password"),
    )
    with psycopg.connect(**dsn) as conn:
        with conn.cursor() as cur:
            cur.execute(inp["sql"], inp.get("params"))
            if cur.description:
                cols = [c.name for c in cur.description]
                rows = cur.fetchall()
                return {"columns": cols, "rows": rows}
            return {"rowcount": cur.rowcount}

def t_run_code(inp: Dict[str,Any]) -> Any:
    code = inp["code"]
    g = {"__builtins__": __builtins__, "time": time}
    l = {}
    exec(code, g, l)  # for lab: do not run untrusted code in prod
    return {"locals": {k: repr(v)[:1000] for k,v in l.items()}}

def t_read(inp: Dict[str,Any]) -> Any:
    assert requests is not None, "requests not installed"
    r = requests.get(inp["url"], timeout=float(inp.get("timeout",20)))
    return {"status": r.status_code, "text": r.text[:20000]}

def _llm_small():
    return ChatOpenAI(model=os.getenv("EXECUTOR_MODEL","gpt-4o-mini"), temperature=0)

def t_summarize(inp: Dict[str,Any]) -> Any:
    llm = _llm_small()
    p = ChatPromptTemplate.from_messages([
        ("system","Summarize precisely for a technical audience."),
        ("human","Summarize:\n\n{c}")
    ])
    msg = (p|llm).invoke({"c": inp.get("content","")})
    return {"summary": msg.content}

def t_analyze(inp: Dict[str,Any]) -> Any:
    llm = _llm_small()
    p = ChatPromptTemplate.from_messages([
        ("system","Analyze and list 3–5 key findings + actions."),
        ("human","Analyze:\n\n{c}")
    ])
    msg = (p|llm).invoke({"c": inp.get("content","")})
    return {"analysis": msg.content}

def t_visualize(inp: Dict[str,Any]) -> Any:
    import io, matplotlib
    matplotlib.use("Agg")
    import matplotlib.pyplot as plt
    x, y = inp["x"], inp["y"]
    fig = plt.figure()
    plt.plot(x, y)
    plt.title(inp.get("title","Plot")); plt.xlabel(inp.get("xlabel","x")); plt.ylabel(inp.get("ylabel","y"))
    buf = io.BytesIO(); fig.savefig(buf, format="png", bbox_inches="tight"); plt.close(fig); buf.seek(0)
    return {"image/png": buf.getvalue()}

TOOLBOX: Dict[str, Callable[[Dict[str,Any]], Any]] = {
    "call_api": t_call_api,
    "query_db": t_query_db,
    "run_code": t_run_code,
    "search": t_read, "read": t_read,
    "summarize": t_summarize, "analyze": t_analyze,
    "visualize": t_visualize,
}

def _exec_step(step: PlanStep) -> StepResult:
    start = time.time()
    try:
        tool_name = step.tool or step.action
        fn = TOOLBOX.get(tool_name)
        if not fn: raise ValueError(f"no tool for '{tool_name}'")
        out = fn(step.inputs or {})
        return StepResult(step_id=step.id, action=step.action, tool=tool_name, ok=True, output=out, latency_s=time.time()-start)
    except Exception as e:
        return StepResult(step_id=step.id, action=step.action, tool=step.tool, ok=False,
                          error=f"{type(e).__name__}: {e}\n{traceback.format_exc()}",
                          latency_s=time.time()-start)

def execute_plan(plan: Plan) -> ExecutionLog:
    log = ExecutionLog(task=plan.task)
    for st in plan.steps:
        res = _exec_step(st); log.add(res)
        if not res.ok: break
    return log


### Putting them together

In [19]:
# === Planner + Executor together ===

from IPython.display import JSON, display
import json as _json

def run_agentic_task(task: str, context: str = ""):
    print("🔮 planning...")
    plan = plan_task(task, context)
    display(JSON(plan.model_dump(), expanded=False))

    print("⚡ executing...")
    log = execute_plan(plan)
    display(JSON(_json.loads(log.to_json()), expanded=False))
    return plan, log


### Set up the UI

In [20]:
# === Minimal Gradio UI ===
%pip install -q gradio

import gradio as gr, json, traceback

def _extract_png(log_json: dict):
    for s in log_json.get("steps", []):
        out = s.get("output") or {}
        if isinstance(out, dict) and "image/png" in out:
            return out["image/png"]
    return None

def ui_runner(task, context):
    try:
        plan = plan_task(task, context or "")
    except Exception:
        return (f"PLANNER ERROR:\n{traceback.format_exc()}", "", None)
    try:
        log = execute_plan(plan)
    except Exception:
        return (json.dumps(plan.model_dump(), indent=2),
                f"EXECUTOR ERROR:\n{traceback.format_exc()}",
                None)

    plan_json = plan.model_dump()
    log_json  = json.loads(log.to_json())
    png = _extract_png(log_json)
    return (json.dumps(plan_json, indent=2), json.dumps(log_json, indent=2), png)

with gr.Blocks(title="Agentic Planner → Executor") as demo:
    gr.Markdown("## planner → executor (LangChain)\nenter a task; i’ll plan it and run it ✨")
    task_in    = gr.Textbox(label="Task", lines=3, value="Summarize this text: 'Kura Labs agentic lab is running; list next actions.'")
    context_in = gr.Textbox(label="Context (optional)", lines=3, value="No DB or external calls; use summarize/analyze.")
    run_btn    = gr.Button("Plan & Execute ⚡", variant="primary")
    plan_out   = gr.Code(label="Plan (JSON)", language="json")
    log_out    = gr.Code(label="Execution Log (JSON)", language="json")
    img_out    = gr.Image(label="Visualization (if any)", type="numpy")
    run_btn.click(ui_runner, [task_in, context_in], [plan_out, log_out, img_out])


Note: you may need to restart the kernel to use updated packages.


IMPORTANT: You are using gradio version 4.21.0, however version 4.44.1 is available, please upgrade.
--------


### Bonus Challenge:

Now that you've built out your very own research agent, can you think of tools that you could add to enhance it? Check out the available tools from Langchain here: https://python.langchain.com/docs/integrations/tools/

Or can you think of another type of application that you could build using the Plan and Execute chains and tools?